In [1]:
import torch
import torch.nn as nn

In [2]:
class StemBlock(nn.Module):
    
    def __init__(self, in_ch=3, out_ch=16):
        # in_ch: default is 3.
        # out_ch: default is 16.
        super(StemBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_ch, out_ch, kernel_size=3, stride=2, padding=1)
        self.conv2 = nn.Conv2d(out_ch, out_ch//2, kernel_size=1, stride=1)
        self.conv3 = nn.Conv2d(out_ch//2, out_ch, kernel_size=3, stride=2, padding=1)
        self.conv4 = nn.Conv2d(out_ch * 2, out_ch, kernel_size=3, stride=1, padding=1)
        
        self.maxPooling = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        self.bn1 = nn.BatchNorm2d(num_features=out_ch)
        self.bn2 = nn.BatchNorm2d(num_features=out_ch//2)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(self.bn1(x))
        
        # x1
        x1 = self.conv2(x)
        x1 = self.relu(self.bn2(x1))
        x1 = self.conv3(x1)
        # [H/4, W/4, C]
        x1 = self.relu(self.bn1(x1))
        
        # x2
        # [H/4, W/4, C]
        x2 = self.maxPooling(x)
        
        x = torch.cat((x1, x2), 1)
        x = self.conv4(x)
        x = self.relu(self.bn1(x))
        return x

In [3]:
class ContextEmbeddingBlock(nn.Module):
    
    def __init__(self, in_ch=128, out_ch=128, stride=1):
        
        super(ContextEmbeddingBlock, self).__init__()
        self.gapPooling = nn.AdaptiveAvgPool2d((1,1))
        self.bn = nn.BatchNorm2d(num_features=in_ch)
        self.conv1 = nn.Conv2d(in_ch, out_ch, kernel_size=1, stride=1)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv2d(in_ch, out_ch, kernel_size=3, stride=1, padding=1)
        
    def forward(self, x):
        x1 = self.gapPooling(x)
        # [None, C, 1, 1]
        if not x1.size()[0] == 1:
            x1 = self.bn(x1)
        x1 = self.conv1(x1)
        if not x1.size()[0] == 1:
            x1 = self.relu(self.bn(x1))
        else:
            x1 = self.relu(x1)
        
        x = torch.add(x1, x)
        x = self.conv2(x)
        return x

In [4]:
class DWConv(nn.Module):
    
    def __init__(self, in_ch, out_ch, stride=1):
        super(DWConv, self).__init__()
        self.depth_conv = nn.Conv2d(in_ch, in_ch, 
                                    kernel_size=3, 
                                    stride=stride,
                                    padding=1,
                                    groups=in_ch)
        
        self.point_conv = nn.Conv2d(in_ch, out_ch, 
                                    kernel_size=1, 
                                    stride=1, 
                                    groups=1)
        
    def forward(self, x):
        x = self.depth_conv(x)
        x = self.point_conv(x)
        return x


class GatherAndExpansionLayer(nn.Module):
    
    def __init__(self, in_ch, out_ch, stride=1):
        super(GatherAndExpansionLayer, self).__init__()
        error = "GatherAndExpansionLayer's stride only support 1 or 2."
        assert stride in [1, 2], error
        self.stride = stride
        if self.stride == 2:
            self.conv2_1 = nn.Conv2d(in_ch, in_ch, kernel_size=3, stride=1, padding=1)
            self.conv2_2 = DWConv(in_ch, 6 * in_ch, stride=2)
            self.conv2_3 = DWConv(6 * in_ch, 6 * in_ch, stride=1)
            self.conv2_4 = nn.Conv2d(6 * in_ch, out_ch, kernel_size=1)
            self.conv2_5 = DWConv(in_ch, out_ch, stride=2)
            self.conv2_6 = nn.Conv2d(out_ch, out_ch, kernel_size=1, stride=1)
        else:
            self.conv1_1 = nn.Conv2d(in_ch, in_ch, kernel_size=3, stride=self.stride, padding=1)
            self.conv1_2 = DWConv(in_ch, 6 * in_ch, stride=self.stride)
            self.conv1_3 = nn.Conv2d(6 * in_ch, out_ch, kernel_size=1, stride=self.stride)
        self.bn1 = nn.BatchNorm2d(num_features=in_ch)
        self.bn2 = nn.BatchNorm2d(num_features=in_ch * 6)
        self.bn3 = nn.BatchNorm2d(num_features=out_ch)
        self.relu = nn.ReLU()
            
    def forward(self, x):
        if self.stride == 2:
            x1 = self.conv2_1(x)
            x1 = self.relu(self.bn1(x1))
            x1 = self.conv2_2(x1)
            x1 = self.bn2(x1)
            x1 = self.conv2_3(x1)
            x1 = self.bn2(x1)
            x1 = self.conv2_4(x1)
            x1 = self.bn3(x1)
            
            x2 = self.conv2_5(x)
            x2 = self.bn3(x2)
            x2 = self.conv2_6(x2)
            x2 = self.bn3(x2)
        else:
            x1 = self.conv1_1(x)
            x1 = self.relu(self.bn1(x1))
            x1 = self.conv1_2(x1)
            x1 = self.bn2(x1)
            x1 = self.conv1_3(x1)
            x1 = self.bn3(x1)
            
            x2 = x
            
        x = torch.add(x1, x2)
        x = self.relu(x)
        return x

In [5]:
class DetailBranch(nn.Module):
    
    def __init__(self):
        super(DetailBranch, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=2, padding=1)
        self.conv2 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(64, 64, kernel_size=3, stride=2, padding=1)
        self.conv4 = nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1)
        self.conv5 = nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1)
        
        self.bn1 = nn.BatchNorm2d(num_features=64)
        self.bn2 = nn.BatchNorm2d(num_features=128)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        # S1
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.relu(self.bn1(self.conv2(x)))
        # S2
        x = self.relu(self.bn1(self.conv3(x)))
        x = self.relu(self.bn1(self.conv2(x)))
        x = self.relu(self.bn1(self.conv2(x)))
        # S3
        x = self.relu(self.bn2(self.conv4(x)))
        x = self.relu(self.bn2(self.conv5(x)))
        x = self.relu(self.bn2(self.conv5(x)))
        
        return x

In [6]:
class SemanticBranch(nn.Module):
    
    def __init__(self, is_train=False):
        super(SemanticBranch, self).__init__()
        self.is_train = is_train
        # S1 and S2
        self.stem = StemBlock(in_ch=3, out_ch=16)
        # S3
        self.ge1 = GatherAndExpansionLayer(in_ch=16, out_ch=32, stride=2)
        self.ge2 = GatherAndExpansionLayer(in_ch=32, out_ch=32, stride=1)
        # S4
        self.ge3 = GatherAndExpansionLayer(in_ch=32, out_ch=64, stride=2)
        self.ge4 = GatherAndExpansionLayer(in_ch=64, out_ch=64, stride=1)
        # S5
        self.ge5 = GatherAndExpansionLayer(in_ch=64, out_ch=128, stride=2)
        self.ge6 = GatherAndExpansionLayer(in_ch=128, out_ch=128, stride=1)
        self.ce = ContextEmbeddingBlock(in_ch=128, out_ch=128, stride=1)
        
    def forward(self, x):
        # S1 and S2
        out1 = self.stem(x)
        # out1: [None, 16, 128, 256]
        
        # S3
        x = self.ge1(out1)
        out2 = self.ge2(x)
        # out2: [None, 32, 64, 128]
        
        # S4
        x = self.ge3(out2)
        out3 = self.ge4(x)
        # out3: [None, 64, 32, 64]

        # S5
        x = self.ge5(out3)
        x = self.ge6(x)
        x = self.ge6(x)
        out4 = self.ge6(x)
        # out4: [None, 128, 16, 32]

        # x: [None, 128, 16, 32]
        x = self.ce(out4)
        if self.is_train:
            return x, out1, out2, out3, out4
        else:
            return x

In [7]:
class SegHeadBooster(nn.Module):
    
    def __init__(self, in_ch, t_ch, out_ch, size):
        super(SegHeadBooster, self).__init__()
        self.conv1 = nn.Conv2d(in_ch, t_ch, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(t_ch, out_ch, kernel_size=1, stride=1, bias=False)
        self.bn = nn.BatchNorm2d(num_features=t_ch)
        self.relu = nn.ReLU()
        self.size = size
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(self.bn(x))
        x = self.conv2(x)
        x = nn.functional.interpolate(x, size=self.size, mode='bilinear')
        return x

In [8]:
class BiseNet_V2(nn.Module):
    
    def __init__(self, num_classes=19, is_train=False):
        super(BiseNet_V2, self).__init__()
        self.is_train = is_train
        # Common
        self.dw = DWConv(in_ch=128, out_ch=128, stride=1)
        self.bn = nn.BatchNorm2d(num_features=128)
        self.conv1_1 = nn.Conv2d(128, 128, kernel_size=1, stride=1)
        self.sigmoid = nn.Sigmoid()
        
        # Detail Branch
        self.detailBranch = DetailBranch()
        self.conv3_3_2 = nn.Conv2d(128, 128, kernel_size=3, stride=2, padding=1)
        self.avgPooling = nn.AvgPool2d(kernel_size=3, stride=2, padding=1)
        
        # Semantic Branch
        self.semanticBranch = SemanticBranch(is_train=self.is_train)
        self.conv3_3_1 = nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1)
        
        # Seg Head
        size = (512, 1024)
        self.segHead0 = SegHeadBooster(in_ch=128, t_ch=64, out_ch=num_classes, size=size)
        self.segHead1 = SegHeadBooster(in_ch=16, t_ch=64, out_ch=num_classes, size=size)
        self.segHead2 = SegHeadBooster(in_ch=32, t_ch=64, out_ch=num_classes, size=size)
        self.segHead3 = SegHeadBooster(in_ch=64, t_ch=64, out_ch=num_classes, size=size)
        self.segHead4 = SegHeadBooster(in_ch=128, t_ch=64, out_ch=num_classes, size=size)
        
        # init weight
        self.init_weight()
        
    def forward(self, x):
        # Detail Branch
        dx = self.detailBranch(x)
        dx1 = self.bn(self.dw(dx))
        dx1 = self.conv1_1(dx1)
        
        dx2 = self.bn(self.conv3_3_2(dx))
        dx2 = self.avgPooling(dx2)
        
        # Semantic Branch
        if self.is_train:
            sx, out1, out2, out3, out4 = self.semanticBranch(x)
            out1 = self.segHead1(out1)
            out2 = self.segHead2(out2)
            out3 = self.segHead3(out3)
            out4 = self.segHead4(out4)
        else:
            sx = self.semanticBranch(x)
        sx1 = self.bn(self.dw(sx))
        sx1 = self.sigmoid(self.conv1_1(sx1))
        
        sx2 = self.bn(self.conv3_3_1(sx))
        sx2 = nn.functional.interpolate(sx2, scale_factor=4, mode='bilinear')
        sx2 = self.sigmoid(sx2)
        
        # Aggregation Layer: element-wise product
        d_out = torch.mul(dx1, sx2)
        s_out = torch.mul(dx2, sx1)
        
        s_out_ = nn.functional.interpolate(s_out, scale_factor=4, mode='bilinear')
        
        out = d_out.add(s_out_)
        # out: [None, 128, 64, 128]
        out0 = self.bn(self.conv3_3_1(out))
        out0 = self.segHead0(out0)
        if self.is_train:
            return out0, out1, out2, out3, out4
        else:
            return out0
    
    def init_weight(self):
        # Kaiming Normal
        for ly in self.children():
            if isinstance(ly, nn.Conv2d):
                nn.init.kaiming_normal_(ly.weight, a=1)
                if not ly.bias is None: nn.init.constant_(ly.bias, 0)

In [9]:
from cityscapes import CityScapes
from torch.utils.data import DataLoader
from tensorboardX import SummaryWriter
from loss import OhemCELoss
from tqdm import tqdm
import os

In [10]:
def poly_lr_scheduler(optimizer, init_lr, iter, lr_decay_iter=1,
                      max_iter=300, power=0.9):
    
    '''Copy from ooooverflow/BiSeNet
    Args:
        init_lr: is base learning rate
        iter: is a current iteration
        lr_decay_iter: how frequently decay occurs, default is 1
        max_iter: is number of maximum iterations
        power: is a polymomial power
    '''
    lr = init_lr * (1 - iter/max_iter) ** power
    optimizer.param_groups[0]['lr'] = lr
    return lr

In [11]:
data_dir = '/media/commaai-03/Data/dataset/Cityscapes'
save_dir = '/home/commaai-03/Mikoy/learn/DeepLearning/pytorch/papers/BiseNet_V2/models'
learning_rate = 5e-2
num_epochs = 10000
batch_size = 16
thresh = 0.7
n_mid = (1024 * 512) // 2

In [12]:
ds_train = CityScapes(data_dir, cropsize=(2048, 1024), mode='train')
dataloader_train = DataLoader(
    ds_train,
    batch_size=4,
    shuffle=False,
    num_workers=4,
    drop_last=True)
ds_val = CityScapes(data_dir, cropsize=(2048, 1024), mode='val')
dataloader_val = DataLoader(
    ds_val,
    batch_size=1,
    shuffle=False,
    num_workers=8)

In [ ]:
diter = iter(dataloader_train)
im, lb = next(diter)

In [ ]:
im.shape

In [13]:
net = BiseNet_V2(is_train=True)
net = torch.nn.DataParallel(net).cuda()

optimizer = torch.optim.SGD(params=net.parameters(),
                            lr=learning_rate,
                            momentum=0.9,
                            weight_decay=5e-4)


writer = SummaryWriter(log_dir='./logs/', comment='Bv2_Batch_16')
loss_func = OhemCELoss(thresh=thresh,
                       n_min=n_mid,
                       ignore_lb=255)

max_miou = 0
step = 0
for epoch in range(num_epochs):
    lr = poly_lr_scheduler(optimizer, learning_rate, iter=epoch, max_iter=num_epochs)
    net.train()
    tq = tqdm(total=len(dataloader_train) * batch_size)
    tq.set_description('[epoch]: %d, [lr]: %f' %(epoch, lr))
    loss_record = []
    for i, (data, label) in enumerate(dataloader_train):
        data = data.cuda()
        label = label.cuda()
        label = torch.squeeze(label, 1)
        optimizer.zero_grad()
        out0, out1, out2, out3, out4 = net(data)
        loss0 = loss_func(out0, label)
        loss1 = loss_func(out1, label)
        loss2 = loss_func(out2, label)
        loss3 = loss_func(out3, label)
        loss4 = loss_func(out4, label)
        loss = loss0 + loss1 + loss2 + loss3 + loss4
        tq.update(batch_size)
        tq.set_postfix(loss='%.5f' % loss)
        
        loss.backward()
        optimizer.step()
        step += 1
        writer.add_scalar('loss_step', loss, step)
        loss_record.append(loss.item())
    
    tq.close()
    loss_avg = np.mean(loss_record)
    writer.add_scalar('epoch/loss_epoch_train', float(loss_avg), epoch)
    print('[Info]: Loss for train: %.5f' % loss_avg)
    if epoch % 50 == 0 and epoch != 0:
        if not os.path.isdir(save_dir):
            os.mkdir(save_dir)
        torch.save(net.module.state_dict(),
                   os.path.join(save_dir, 'latest_dice_loss.pth'))
    
    # Validation after

[epoch]: 0, [lr]: 0.050000:   0%|          | 0/11888 [00:00<?, ?it/s]/home/commaai-03/.local/lib/python3.6/site-packages/torch/nn/functional.py:2506: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))
[epoch]: 1, [lr]: 0.049995:   0%|          | 0/11888 [00:00<?, ?it/s]

[Info]: Loss for train: 8.05211


[epoch]: 2, [lr]: 0.049991:   0%|          | 0/11888 [00:00<?, ?it/s]

[Info]: Loss for train: 7.39447


[epoch]: 3, [lr]: 0.049986:   0%|          | 0/11888 [00:00<?, ?it/s]

...

[Info]: Loss for train: 6.49478


[epoch]: 221, [lr]: 0.049004:   0%|          | 0/11888 [00:00<?, ?it/s]

[Info]: Loss for train: 6.58070


[epoch]: 221, [lr]: 0.049004:  92%|█████████▏| 10912/11888 [04:00<00:21, 45.04it/s, loss=6.36852]

KeyboardInterrupt: 

In [ ]:
?SummaryWriter